In [ ]:
import pandas as pd
import itertools as it
import string
from collections import Counter
import json
import re
import numpy
import spacy

##### Load Sentence Splitting and POS Tag Models

In [ ]:
with open('data/stopwords.txt') as fh:
    stopwords = set(fh.read().splitlines())

In [ ]:
spacy_nlp = spacy.load("en_core_web_lg")

##### Define various helper functions

In [ ]:
def normalise(text):
    
    text = text.lower()
    text = text.replace("\n", " ")
    text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ", text)
    text = text.replace("/", " / ")
    text = re.sub(r"@\w+", " <user> ", text)
    text = re.sub(r"[8:=;]['`\-][)d]+|[)d]+['`\-][8:=;]", " <smile> ", text)
    text = re.sub(r"[8:=;]['`\-]p+", " <lolface> ", text)
    text = re.sub(r"[8:=;]['`\-]\(+|\)+['`\-][8:=;]", " <sadface> ", text)
    text = re.sub(r"[8:=;]['`\-][\/|l*]", " <neutralface> ", text)
    text = text.replace(r"<3", " <heart> ")
    text = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ", text)
    text = text.replace(r"#", " <hashtag> ")
    text = re.sub(r"([!?.,()])+", r" \1 ", text)
    
    return text

In [ ]:
def tokenise(text):
    return [t.strip() for t in text.strip().split()]

In [ ]:
def parse_treatment_definitons(definion_file):
    treatment_set = set()
    treatment_mapping = {}
    max_length = 1
    for line in definion_file:
        line = line.strip()
        treatments = line.split(',')
        name = treatments[0]
        for treatment in treatments:
            treatment = tuple(tokenise(normalise(treatment)))
            max_length = max(len(treatment), max_length)
            treatment_set.add(treatment)
            treatment_mapping[treatment] = name
    return treatment_set, treatment_mapping, max_length

In [ ]:
def window_sliding(iterable, n):
    gens = (
        it.chain(it.repeat(None, n - 1 - i), iterable, it.repeat(None, i))
        for i, gen in enumerate(it.tee(iterable, n)))
    return zip(*gens)

def find_treatments(tokens):
    found_treatments = []
    for x in range(max_treatment_length, 0, -1):
        for window in window_sliding(tokens, x):
            if tuple(window) in treatment_set:
                found_treatments.append(treatment_mapping[tuple(window)])
    return list(set(found_treatments)) or None

In [ ]:
def filter_stopwords(tokens):
    return [t for t in tokens if len(t) > 2 and t not in stopwords]

In [ ]:
def calculate_embeddings(tokens):
    token_embdgs = [embeddings.get(t) for t in tokens]
    token_embdgs = [t for t in token_embdgs if t is not None]
    return numpy.mean(token_embdgs, axis=0)

In [ ]:
def add_pos_tags(df):
    tokens = []
    pos = []
    tag = []

    for doc in spacy_nlp.pipe(df['text'].astype('unicode').values, batch_size=50, n_threads=3):
        if doc.is_parsed:
            tokens.append([n.text for n in doc])
            pos.append([n.pos_ for n in doc])
            tag.append([n.tag_ for n in doc])
        else:
            tokens.append(None)
            pos.append(None)
            tag.append(None)

    df['tokens'] = tokens
    df['pos'] = pos
    df['tag'] = tag
    return df

##### Load Tweets into DataFrame

In [ ]:
with open('data/tweets.jsonl', 'r') as fh:
    df = pd.read_json(fh.read(), lines=True, convert_dates=True)

##### Filter retweets, non-English tweets and tweets outside of investigation period

In [ ]:
df = df.loc[(df['retweeted_status'].isnull()) & (df['lang'] == "en")]

In [ ]:
df = df.loc[(df.created_at >= "2017-12-04") & (df.created_at <= "2018-06-30")]

###### Do some Twitter-specific normalisations of text and username

In [ ]:
df["extended_tweet"] = df["extended_tweet"].apply(lambda x: x["full_text"]  if type(x) == dict else x )

In [ ]:
df["text"] = df["extended_tweet"].combine_first(df["text"])

In [ ]:
df["text"] = df["text"].apply(lambda x: normalise(x).replace("\n", " "))

In [ ]:
df["user_name"] = df["user"].apply(lambda x: x["screen_name"])

In [ ]:
df = df.loc[:, ["id", "created_at", "text", "retweet_count", "user_name"]]

##### Get some statistics on the full dataset

In [ ]:
len(df.index)  # number of total tweets

In [ ]:
len(df.user_name.unique()) # number of users writing these tweets

##### Get POS-Tags, create POS frequencies and store them

In [ ]:
df = add_pos_tags(df)

In [ ]:
noun_tags = {"NN", "NNS", "NNP", "NNPS"}
adj_tags = {"JJ", "JJR", "JJS"}
nouns = Counter()
adjectives = Counter()
for _, row in df.iterrows():
    tokens = row["tokens"]
    tags = row["tag"]
    for cnt, tag in enumerate(tags):
        if tag in noun_tags:
            nouns[tokens[cnt]] += 1
        if tag in adj_tags:
            adjectives[tokens[cnt]] += 1

In [ ]:
sum(nouns.values())  # number of noun types

In [ ]:
len(nouns)  # number of noun tokens

In [ ]:
nouns.most_common(10)  # most frequent nouns

In [ ]:
sum(adjectives.values()) # number of adjective types

In [ ]:
len(adjectives)  # number of adjective tokens

In [ ]:
adjectives.most_common(10)  # most frequent adjectives

In [ ]:
with open("data/nouns_twitter.json", 'w') as fh:
    json.dump(nouns, fh)
with open("data/adjectives_twitter.json", 'w') as fh:
    json.dump(adjectives, fh)

##### Tokenize Text

In [ ]:
df["tokens"] = df["text"].apply(tokenise)

##### Find mentioned treatments, filter tweets without

In [ ]:
treatment_set, treatment_mapping, max_treatment_length = parse_treatment_definitons(open("data/treatment_definitons.txt", 'r'))

In [ ]:
df['treatments'] = df['tokens'].apply(find_treatments)

In [ ]:
df = df.loc[df['treatments'].notnull()]

In [ ]:
len(df.index)  # number of tweets with treatment

In [ ]:
len(df.user_name.unique())  # number of authors of these tweets

##### Store result

In [ ]:
df.to_csv("tweets_with_treatments.csv")

##### Create treatment frequencies, store these

In [ ]:
treatment_counts = Counter()
for elem in df["treatments"]:
    treatment_counts += Counter(elem)

In [ ]:
with open("data/treatments_twitter.json", 'w') as fh:
    json.dump(treatment_counts, fh)